In [ ]:
import pandas as pd
import glob
import os
import shutil
import re
from joblib import Parallel, delayed
import nibabel as nib
import numpy as np
import random
import seaborn as sns
from datetime import datetime

In [ ]:
import sys
sys.path.append('/media/raid/orco/data/ICH/code/neurorad_bids_utils')
from bids_utils.utils import utils, mask_utils

In [ ]:
dataset_name = 'Dataset011_5FoldCV'

In [ ]:
preds = glob.glob('/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Dataset01*CV/test_fold_1/OrbitAI_*.nii.gz')
len(preds)

In [ ]:
df = pd.DataFrame(preds)
df.columns = ['prediction']
df['gt'] = df.prediction.apply(lambda x: '/home/orco/data/Orbita_AI/Dataset/labels_231216/' + os.path.basename(x).replace('OrbitAI_',''))
df['fold'] = df.prediction.apply(lambda x: x.split('/')[-2].split('_')[-1])

In [ ]:
res = []
for i,j in zip(list(df.prediction),list(df['gt'])):
    res.append([i,j] + utils.mp_vol_nii_n(i, [1,2,3,4], n_jobs=4) + utils.mp_vol_nii_n(j, [1,2,3,4], n_jobs=4) + [mask_utils.has_label(j,3), mask_utils.has_label(j,4), mask_utils.has_label(i,3), mask_utils.has_label(i,4)])

In [ ]:
df = pd.DataFrame(res)

In [ ]:
df.head()

In [ ]:
df.columns = ['prediction', 'gt', 'Vol_pred_eye', 'Vol_pred_nerve', 'Vol_pred_tumor', 'Vol_pred_ret_det', 'Vol_gt_eye', 'Vol_gt_nerve', 'Vol_gt_tumor', 'Vol_gt_ret_det', 'has_tumor', 'has_ret_det', 'pred_tumor', 'pred_ret_det']


In [ ]:
df['sid'] = df['prediction'].apply(lambda x: x.split('_')[-1].split('.')[0])
df['dataset'] = df['prediction'].apply(lambda x: x.split('/')[-3])#.split('.')[0])

In [ ]:
df['dataset']

In [ ]:
date = '{}{}{}'.format(datetime.now().year, datetime.now().month, datetime.now().day)
date

In [ ]:
df.to_csv('../data/volumes_{}_{}.csv'.format('5FoldCV', date), index=False)